In [32]:
from clickhouse_driver import Client
import json 
from pprint import pprint

client = Client(host='localhost', user='default', password='', port=9000)         

try:
    result = client.execute('SHOW TABLES')
    print(result)
except Exception as e:
    print(f"Ошибка работы с базой данных: {e}")

[]


In [2]:
# Создаем БД
client.execute('CREATE DATABASE IF NOT EXISTS books2scrape')

[]

In [33]:
# Удаление таблицы (для отладки)
# client.execute('DROP TABLE books2scrape.books')

[]

In [34]:
# Создание основной таблицы 'books' с автоматической генерацией уникального идентификатора
client.execute('''
CREATE TABLE IF NOT EXISTS books2scrape.books (
    id UUID DEFAULT generateUUIDv4(),
    title String,
    price String,
    available UInt16,
    description String
) 
ENGINE = MergeTree()
ORDER BY id
''')

print("Таблица books создана успешно.")
client.execute('SHOW TABLES')

Таблица books создана успешно.


[]

In [35]:
# Чтение файла JSON
with open('books.json', 'r', encoding='utf-8') as file:
    data = json.load(file)
print(len(data))

# Приводим имена полей к нижнему регистру
for i in range(len(data)):    
    data[i] = {key.lower(): value for key, value in data[i].items()}
print(data[0])

1000
{'title': 'A Light in the Attic', 'price': '£51.77', 'available': 22, 'description': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock?And who put you up there,And y

In [37]:
# Вставка данных в таблицу
client.execute("""
    INSERT INTO books2scrape.books (title, price, available, description)
    VALUES""", data)

print("Данные введены успешно.")

# Проверка успешности вставки
result = client.execute("SELECT * FROM books2scrape.books")
print("Первая вставленная запись:", result[0])

result = client.execute('SELECT COUNT(*) FROM books2scrape.books')
print(f"Всего записей в таблице: {result[0][0]}")

Данные введены успешно.
Первая вставленная запись: (UUID('4fa9fac8-7aa4-4bc2-8002-d2c77f1654b1'), 'The Gray Rhino: How to Recognize and Act on the Obvious Dangers We Ignore', '£59.15', 12, 'A "gray rhino" is a highly probable, high impact yet neglected threat: kin to both the elephant in the room and the improbable and unforeseeable black swan. Gray rhinos are not random surprises, but occur after a series of warnings and visible evidence. The bursting of the housing bubble in 2008, the devastating aftermath of Hurricane Katrina and other natural disasters, t A "gray rhino" is a highly probable, high impact yet neglected threat: kin to both the elephant in the room and the improbable and unforeseeable black swan. Gray rhinos are not random surprises, but occur after a series of warnings and visible evidence. The bursting of the housing bubble in 2008, the devastating aftermath of Hurricane Katrina and other natural disasters, the new digital technologies that upended the media world, t

In [52]:
# Использование LIKE для фильтрации по наличию слова 'music' в поле 'title'
result = client.execute("""
    SELECT * FROM books2scrape.books    
    WHERE LOWER(title) LIKE '%music%'
""")

for row in result:
    print(row)

(UUID('fc149fb7-a5c8-436e-8e83-34fbb90db6ec'), 'Musicophilia: Tales of Music and the Brain', '£46.58', 6, 'What goes on in human beings when they make or listen to music? What is it about music, what gives it such peculiar power over us, power delectable and beneficent for the most part, but also capable of uncontrollable and sometimes destructive force? Music has no concepts, it lacks images; it has no power of representation, it has no relation to the world. And yet it is evi What goes on in human beings when they make or listen to music? What is it about music, what gives it such peculiar power over us, power delectable and beneficent for the most part, but also capable of uncontrollable and sometimes destructive force? Music has no concepts, it lacks images; it has no power of representation, it has no relation to the world. And yet it is evident in all of usâ\x80\x93we tap our feet, we keep time, hum, sing, conduct music, mirror the melodic contours and feelings of what we hear in

In [51]:
# Использование операторов сравнения (фильтруем по наличию)
result = client.execute("""
    SELECT * FROM books2scrape.books    
    WHERE available > 20 AND available < 30
""")

for row in result:
    print(row)

(UUID('1dbe3866-68f0-4ec8-b031-99b197cc2269'), 'A Light in the Attic', '£51.77', 22, "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock?And who put you up there,And your c